In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.cluster import KMeans
from sklearn.gaussian_process.kernels import Matern, RBF

import plotly.express as px

import plotly.graph_objects as go

from scipy.linalg import toeplitz, block_diag

from spe.mse_estimator import ErrorComparer
from spe.data_generation import gen_rbf_X, gen_matern_X, create_clus_split, gen_cov_mat
from spe.forest import BlurredForest
from spe.estimators import kfoldcv, kmeanscv, better_test_est_split, cp_rf_train_test, cp_general_train_test, bag_kfoldcv, bag_kmeanscv

import os

In [182]:
niter = 50

n=30**2
p=30
s=30

k=2

delta = 0.8

snr = 1.

n_estimators = 50
max_depth = 6

noise_kernel = 'matern'
noise_length_scale = 5.
noise_nu = 0.5

X_kernel = 'matern'
X_length_scale = 5.
X_nu = 0.5

idx = -1

savedir='~'

In [183]:
# noise_kernel = kernel
# X_kernel = 'matern'#kernel

# noise_length_scale = length_scale
# X_length_scale = 1.#length_scale

# noise_nu = nu
# X_nu = 1.5#nu

In [184]:
models = BlurredForest(n_estimators=n_estimators,  
                        max_depth=max_depth, 
                        bootstrap_type='blur')

ests = [better_test_est_split,
        cp_rf_train_test, 
        cp_rf_train_test, 
        bag_kfoldcv,
        bag_kmeanscv]

est_kwargs = [{'full_refit':True},
              {'use_trace_corr': False, 'full_refit':True}, 
              {'use_trace_corr': True, 'full_refit':True},
              {'k': k},
              {'k': k}]

In [185]:
if not os.path.exists(os.path.expanduser(savedir)):
    os.makedirs(os.path.expanduser(savedir))

params = pd.DataFrame({'niter': niter,
                       'n': n, 
                       'p': p, 
                       's': s,
                       'k': k,
                       'snr':snr,
                       'n_estimators': n_estimators,
                       'max_depth': max_depth,
                       'delta': delta,
                       'nk': noise_kernel, 
                       'nls': noise_length_scale, 
                       'nn': noise_nu, 
                       'xk': X_kernel,
                       'xls': X_length_scale,
                       'xn': X_nu}, index=[idx])
params.to_csv(os.path.expanduser(savedir + 'params.csv'))
dffp = os.path.expanduser(savedir + "err_df.csv")
# barfp = os.path.expanduser(savedir + 'barchart.jpeg')

In [186]:
err_cmp = ErrorComparer()

In [187]:
nx = ny = int(np.sqrt(n))
xs = np.linspace(0, 10, nx)
ys = np.linspace(0, 10, ny)
c_x, c_y = np.meshgrid(xs, ys)
c_x = c_x.flatten()
c_y = c_y.flatten()
coord = np.stack([c_x, c_y]).T

In [188]:
if noise_kernel == 'rbf':
    Sigma_t = gen_cov_mat(c_x, c_y, RBF(length_scale=noise_length_scale))
elif noise_kernel == 'matern':
    Sigma_t = gen_cov_mat(c_x, c_y, Matern(length_scale=noise_length_scale, nu=noise_nu))
else:
    Sigma_t = np.eye(n)
    
Sigma_t = delta*Sigma_t + (1-delta)*np.eye(n)

if noise_kernel == 'rbf' or noise_kernel == 'matern':
    Chol_t = np.linalg.cholesky(Sigma_t)
else:
    Chol_t = np.eye(n)

In [189]:
if X_kernel == 'rbf':
    X = gen_rbf_X(c_x, c_y, p)
elif X_kernel == 'matern':
    X = gen_matern_X(c_x, c_y, p, length_scale=X_length_scale, nu=X_nu)
else:
#     X = np.random.randn(n,p)
    X = np.random.uniform(size=(n,p))

beta = np.zeros(p)
idx = np.random.choice(p,size=s)
beta[idx] = np.random.uniform(-1,1,size=s)

In [190]:
(test_err,
 rnd_err,
 trc_err,
 kfcv_err,
 spcv_err) = err_cmp.compare(models,
                             ests,
                             est_kwargs,
                             niter=niter,
                             n=n,
                             p=p,
                             s=p,
                             snr=snr, 
                             X=X,
                             beta=beta,
                             coord=coord,
                             Chol_t=Chol_t,
                             Chol_s=None,
                             tr_idx=None,#np.ones(n, dtype=bool),
                             fair=False,
                             )

0
0.5833333333333334
10
20
30
40


In [191]:
risk = test_err.mean()
risk_kfcv = kfcv_err.mean()
risk_spcv = spcv_err.mean()
risk_rnd = rnd_err.mean()
risk_trc = trc_err.mean()
# risk, risk_kfcv, risk_spcv, risk_lin

In [192]:
save_df = pd.DataFrame({'Rand_FR': (rnd_err.T),
                        'Trace_FR': (trc_err.T),
                        'KFCV': (kfcv_err.T),
                        'SPCV': (spcv_err.T),
                        'Test': (test_err.T)})
save_df.to_csv(dffp)

In [193]:
df = pd.DataFrame({'Rand_FR': (rnd_err.T),
                   'Trace_FR': (trc_err.T),
                   'KFCV': (kfcv_err.T),
                   'SPCV': (spcv_err.T)})

In [194]:
(df/risk).mean()

Rand_FR     0.900514
Trace_FR    0.904701
KFCV        0.185654
SPCV        0.883926
dtype: float64

In [195]:
df.var(), (df/risk).var()

(Rand_FR     0.001523
 Trace_FR    0.002067
 KFCV        0.000019
 SPCV        0.016159
 dtype: float64,
 Rand_FR     0.012204
 Trace_FR    0.016568
 KFCV        0.000153
 SPCV        0.129516
 dtype: float64)

In [196]:
fig = go.Figure()
fig.add_trace(go.Bar(
    name='Control',
    x=['Rand_FR', 'Trace_FR', 'KFCV', 'SPCV'], y=(df/risk).mean(),
    marker_color=px.colors.qualitative.Plotly,
    text=np.around((df/risk).mean(),3),
    textposition='outside',
    error_y=dict(
        type='data',
        color='black',
        symmetric=False,
        array=(df/risk).quantile(.75) - (df/risk).mean(),
        arrayminus=(df/risk).mean() - (df/risk).quantile(.25))
#         array=(df/risk).mean() + (df/risk).std(),
#         arrayminus=(df/risk).mean() - (df/risk).std())
))
# fig.add_trace(go.Bar(
#     name='Experimental',
#     x=['GenCp', 'KFCV', 'SPCV'], y=(df/risk).mean(),
#     error_y=dict(type='data', array=[1, 2])
# ))
# fig.update_layout(barmode='group')
fig.add_hline(y=1., line_color='red')
fig.update_layout(
    title=f"BFFR_\u03B4{delta}_snr{snr}_nk{noise_kernel}_nls{noise_length_scale}_nv{noise_nu}_Xk{X_kernel}_Xls{X_length_scale}_Xv{X_nu}",
    xaxis_title="Method",
    yaxis_title="Relative MSE",
#     legend_title="Legend Title",
#     font=dict(
#         family="Courier New, monospace",
#         size=18,
#         color="RebeccaPurple"
#     )
)
barfp = os.path.expanduser(savedir + fig.layout.title['text'] + ".jpeg")
fig.write_image(os.path.expanduser(barfp))
fig.show()